In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# net income before extraordinary items 特别项目前收入
# extraordinary items可以认为是营业外净收入
# 净利润 + 营业外支出 - 营业外收入
# 或：营业利润 - 所得税
df1 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计', '所有者权益合计'], low_memory=False)
df2 = pd.read_csv('../../data/csmar/季_利润表.csv', usecols=['证券代码', '会计期间', '营业利润', '减：所得税费用'])
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3 = diff(df3, col=['营业利润', '减：所得税费用'])
df3['特别项目前收入'] = df3['营业利润'] - df3['减：所得税费用']

# 69.  roaq
# 季度频率。特别项目前收入除以滞后一季度的总资产。
df3['roaq'] = df3['特别项目前收入']/df3.groupby('证券代码').shift(1)['资产总计'].replace(0, np.nan)

# 70.  roavol
# 季度频率。16个季度的特别项目前收入的标准差，除以平均总资产。
df3['平均资产'] = df3[['证券代码', '资产总计']].groupby('证券代码').rolling(window=16).mean().reset_index()['资产总计']
df3['特别项目前收入标准差'] = df3[['证券代码', '特别项目前收入']].groupby('证券代码').rolling(window=16).std().reset_index()['特别项目前收入']
df3['roavol'] = df3['特别项目前收入标准差']/df3['平均资产'].replace(0, np.nan)

# 71.  roeq
# 季度频率。特别项目前收入除以滞后普通股股东权益。
df3['roeq'] = df3['特别项目前收入']/df3.groupby('证券代码').shift(1)['所有者权益合计'].replace(0, np.nan)

# 49.  operprof
# 季度频率。季度营业利润除以滞后一期的普通股股东权益。
df3['operprof'] = df3['营业利润']/df3.groupby('证券代码').shift(1)['所有者权益合计'].replace(0, np.nan)

# 19.  chpm
# 季度频率。特别项目前收入的变化除以总资产。
df3['chpm'] = (df3['特别项目前收入'] - df3.groupby('证券代码').shift(1)['特别项目前收入'])/df3['资产总计'].replace(0, np.nan)

#20.  chpm_ia
#季度频率。行业调整后的 chpm。
df4 = pd.read_csv('../../data/csmar/公司文件.csv', usecols=['证券代码', '行业代码C'])
df5 = pd.merge(df3, df4, on=['证券代码'])
df6 = df5[['行业代码C', '会计期间', 'chpm']].groupby(['行业代码C', '会计期间']).mean().reset_index()
df6.rename(columns={'chpm':'chpm_i'}, inplace=True)
df7 = pd.merge(df5, df6, on=['行业代码C', '会计期间'], how='left')
df7['chpm_ia'] = df7['chpm'] - df7['chpm_i']

df8 = df7[['证券代码', '会计期间', 'chpm', 'chpm_ia', 'operprof', 'roaq', 'roavol', 'roeq']]
df8.to_csv('../../output/csmar/季_19_20_49_69_70_71.csv', index=False, encoding='utf-8-sig')
df8

,证券代码,会计期间,chpm,chpm_ia,operprof,roaq,roavol,roeq
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
728835,900957,2022-03-31,0.000974,0.000026,0.016003,0.007597,NaN,0.014772
728836,900957,2022-06-30,0.006183,0.007367,0.028072,0.013782,NaN,0.026657
728837,900957,2022-09-30,-0.005684,-0.005210,0.016610,0.007900,NaN,0.015058
728838,900957,2022-12-31,0.006007,0.011667,0.033699,0.013936,NaN,0.026344
